In [ ]:
#Reads in wikigold.txt. Converts to spacy formats.
#Predicts tags for wikigold text using spacy pretrained model.
#Calculates precision,recall and F1 using the tags in wikigold.txt as ground truth.
#The code needs to be optimized for computational speed (some for and ifs can be avoided).

In [6]:
#Ideally should be a module in seperate file

#Read in each sentence from wikigold.txt and put it in the tuple format,
#("Trump says he's answered Mueller's Russia inquiry questions \u2013 live",[(0,5,"PERSON"),(25,32,"PERSON")])
def process_one_sentence(fh):

 entlst = []
 wrdlst = []
 sentence = ""
 pos_ent = []

 i=0   
 ln = fh.readline()
 while ln != '\n':
  wrd,ent = ln.strip().split()  
  if ent!='O':
   if ent=='I-ORG':
    ent='ORG'
   if ent=='I-LOC':
    ent='GPE'
   if ent=='I-PER':
    ent='PERSON'   
   if ent=='I-MISC':
    ent='MISC'       
   entlst.append(ent)  
   st = len(sentence)
   if i==0:
    tup = (st,st+len(wrd),ent)
   else: 
    tup = (st+1,st+len(wrd)+1,ent)
   pos_ent.append(tup)  
  wrdlst.append(wrd)   
  sentence = " ".join(wrdlst)
  #if ent!='O':  
  # print(sentence,tup,sentence[tup[0]:tup[1]])  
  ln = fh.readline()
  i+=1

 return sentence,pos_ent,entlst   

In [65]:
import numpy as np

fhr = open('data/wikigold.txt','r',encoding='utf8')

list_tup = []
concat_ent = []

nlines=1841

#Make a list of tuples returned by process_one_sentence()
baddie = 0
j = 0
for i in np.arange(nlines):
 sentc,pelst,entlst = process_one_sentence(fhr)
 concat_ent.append(entlst)
 if pelst:
  tmptup = (sentc,pelst)
  list_tup.append(tmptup)
  j+=1  
 else:
  baddie += 1

print('---# of sentences without any entities = ',baddie,500-baddie,len(list_tup))    
print('\nTotal number of sublists = ',len(concat_ent))

list2 = [x for x in concat_ent if x]
flat_list = [item for xx in list2 for item in xx]
print('\nTotal number of words wiht entities = ',len(flat_list))
 
myset = set(flat_list)

print('\nUnique entities = ',myset)


---# of sentences without any entities =  500 0 1341

Total number of sublists =  1841

Total number of words wiht entities =  6431

Unique entities =  {'ORG', 'GPE', 'PERSON', 'MISC'}


In [58]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

nlp = spacy.load('en_core_web_sm')

In [66]:
#Score the predictions of the spaCy model against ground truth, for each sentence.

def test_score(intext,entity_offs):
 docobj = nlp.make_doc(intext)
 gold = GoldParse(docobj, entities=entity_offs)
 pred_val = nlp(intext)
 
# if i<=70:
# print()
 #for st,ed,true_ent in entity_offsets:
 #print('',raw_text[st:ed],true_ent)      

 #print('')
 #for xtup in pred_val.ents:
 # print('Predicted : %15s %15s'%(xtup.text,xtup.label_))   

 scorer = Scorer()
 scorer.score(pred_val,gold)
 ss = scorer.scores
    
 #if i<=70:
  #print('\n',ss)
  #print('\n\tprecision, recall, fscore = ',ss['ents_p'],ss['ents_r'],ss['ents_f'])
  #print('\tents_per_type = ',ss['ents_per_type'])
    
 return  ss['ents_per_type'] 

In [67]:
#scores are as expected for test_data
#test_data =  [
#              ("Trump says he's answered Mueller's Russia inquiry questions \u2013 live",[(0,5,"PERSON"),(25,32,"PERSON")]),
#              ("Alexander Zverev reaches ATP Finals semis then reminds Lendl who is boss",[(0,16,"PERSON"),(55,60,"PERSON")]),  
#              ("Britain's worst landlord to take nine years to pay off string of fines",[(0,7,"GPE")]),
#              ("Tom Watson: people's vote more likely given weakness of May's position",[(0,10,"PERSON"),(56,59,"PERSON")]),
#            ]

#these entities are present in wikigold.txt and in spaCy model
common_ent = ['ORG','GPE','PERSON']
avg_score = np.empty([3,3],float)
nsamp = np.empty(3,int)
avg_score[:,:] = 0.0
nsamp[:] = 0

#Calcualte average scores over all sentences.
i=0
#for raw_text, entity_offsets in test_data:
for raw_text, entity_offsets in list_tup:
 #print('\n---i = ',i)
 #print('\n---\nraw_text=',raw_text)   
 myscores = test_score(raw_text,entity_offsets)
 #print('myscores = ',myscores)
 #print('values = ',myscores.values())   
 scorelst = list(myscores.keys()) 
 #print('scorelst = ',scorelst)
 for myent in scorelst:
  if myent in common_ent:      
   #print('^^myent = ',myent,myscores[myent]['p'])
   if myent=='ORG':
    avg_score[0,0:3] += myscores[myent]['p'],myscores[myent]['r'],myscores[myent]['f']
    nsamp[0] += 1
   if myent=='GPE':
    avg_score[1,0:3] += myscores[myent]['p'],myscores[myent]['r'],myscores[myent]['f']
    nsamp[1] += 1
   if myent=='PERSON':
    avg_score[2,0:3] += myscores[myent]['p'],myscores[myent]['r'],myscores[myent]['f']
    nsamp[2] += 1
 i+=1


print('\n---avg_scores = ')
i=0
for myent in common_ent:
 print('\nnsamp, myent = ',nsamp[i],myent)   
 print('\tprec, recall, F = ',avg_score[i,:]/nsamp[i])
 i+=1

#Unique entities =  {'I-ORG', 'I-LOC', 'I-PER', 'I-MISC'}
#                       ORG,     GPE,    PERSON, 'MISC'


---avg_scores = 

nsamp, myent =  772 ORG
	prec, recall, F =  [15.62577103 14.32586556 14.00159929]

nsamp, myent =  616 GPE
	prec, recall, F =  [47.00113379 37.6814617  39.78535666]

nsamp, myent =  685 PERSON
	prec, recall, F =  [16.51953628 13.84306463 14.52505849]
